In [ ]:
# --- Cell 1: Import Libraries and Set Up Environment ---
"""
# Handwritten Character Recognition: Interactive Gradio Application

This notebook builds an interactive web application for handwritten character recognition using Gradio.
It allows users to recognize characters through various input methods and visualize the results.
"""

import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
from PIL import Image
import json
from glob import glob
import base64
from io import BytesIO

# PyTorch imports
import torch
import torch.nn.functional as F
from torchvision import transforms

# Import utility modules
from src.models_util import get_model, get_model_info
from src.inference_utils import prepare_single_image, process_numpy_image
from src.inference_utils import extract_characters_from_image, benchmark_inference_speed

# Gradio for UI
import gradio as gr

# For reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

# Device configuration
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# Create directories for saving results
os.makedirs("gradio_results", exist_ok=True)
os.makedirs("gradio_results/uploads", exist_ok=True)
os.makedirs("gradio_results/drawings", exist_ok=True)
os.makedirs("gradio_results/captures", exist_ok=True)
os.makedirs("gradio_results/batch", exist_ok=True)



In [ ]:
# --- Cell 2: Model Loading Functions ---
"""
## Model Loading Functions

These functions handle loading trained models from checkpoints.
They include options for different model architectures and configurations.
"""

def load_class_names(class_names_path):
    """
    Load class names from a text file.
    
    Args:
        class_names_path: Path to the class names file
        
    Returns:
        list: List of class names
    """
    print(f"Loading class names from: {class_names_path}")
    
    if not os.path.exists(class_names_path):
        print(f"ERROR: Class names file '{class_names_path}' not found")
        return None
    
    try:
        with open(class_names_path, 'r') as f:
            class_names = [line.strip() for line in f if line.strip()]
        
        print(f"Loaded {len(class_names)} class names")
        return class_names
    
    except Exception as e:
        print(f"Error loading class names: {e}")
        return None

def find_available_models(model_checkpoints_dir="../model_checkpoints"):
    """
    Find available trained models in the checkpoints directory.
    
    Args:
        model_checkpoints_dir: Directory containing model checkpoints
        
    Returns:
        dict: Dictionary of available models
    """
    print(f"Searching for available models in: {model_checkpoints_dir}")
    
    if not os.path.exists(model_checkpoints_dir):
        print(f"ERROR: Model checkpoints directory '{model_checkpoints_dir}' not found")
        return {}
    
    available_models = {}
    
    # List subdirectories (each should be a model type)
    model_dirs = [d for d in os.listdir(model_checkpoints_dir) 
                if os.path.isdir(os.path.join(model_checkpoints_dir, d))]
    
    for model_dir in model_dirs:
        model_path = os.path.join(model_checkpoints_dir, model_dir)
        
        # Check for model checkpoints
        checkpoint_files = []
        for ext in ['*.pth', '*.pt']:
            checkpoint_files.extend(glob(os.path.join(model_path, ext)))
        
        if checkpoint_files:
            # Check for class_names.txt
            class_names_path = os.path.join(model_path, 'class_names.txt')
            has_class_names = os.path.exists(class_names_path)
            
            # Check for model_info.txt
            model_info_path = os.path.join(model_path, 'model_info.txt')
            has_model_info = os.path.exists(model_info_path)
            
            available_models[model_dir] = {
                'checkpoint_files': sorted(checkpoint_files),
                'class_names_path': class_names_path if has_class_names else None,
                'model_info_path': model_info_path if has_model_info else None,
                'has_class_names': has_class_names,
                'has_model_info': has_model_info
            }
    
    if available_models:
        print(f"Found {len(available_models)} available models:")
        for model_name, info in available_models.items():
            print(f"  - {model_name}:")
            print(f"    - Checkpoints: {len(info['checkpoint_files'])}")
            print(f"    - Has class names: {info['has_class_names']}")
            print(f"    - Has model info: {info['has_model_info']}")
    else:
        print("No trained models found.")
    
    return available_models

def load_model_checkpoint(checkpoint_path, model_name, num_classes, device=device):
    """
    Load a model from a checkpoint file.
    
    Args:
        checkpoint_path: Path to the checkpoint file
        model_name: Name of the model architecture
        num_classes: Number of output classes
        device: Device to load the model on
        
    Returns:
        torch.nn.Module: Loaded model
    """
    print(f"Loading model checkpoint from: {checkpoint_path}")
    
    if not os.path.exists(checkpoint_path):
        print(f"ERROR: Checkpoint file '{checkpoint_path}' not found")
        return None
    
    try:
        # Initialize the model architecture
        model = get_model(model_name, num_classes, device, pretrained=False)
        
        # Load the checkpoint
        checkpoint = torch.load(checkpoint_path, map_location=device)
        
        # Extract state dict
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
        else:
            # Handle case where checkpoint is just the state dict
            state_dict = checkpoint
        
        # Load state dict into model
        model.load_state_dict(state_dict)
        
        # Set model to evaluation mode
        model.eval()
        
        print(f"Model loaded successfully with {sum(p.numel() for p in model.parameters())} parameters")
        
        # Extract metadata if available
        metadata = {}
        for key in ['epoch', 'accuracy', 'val_acc', 'loss']:
            if key in checkpoint:
                metadata[key] = checkpoint[key]
        
        if metadata:
            print(f"Checkpoint metadata:")
            for key, value in metadata.items():
                print(f"  {key}: {value}")
        
        return model
    
    except Exception as e:
        print(f"Error loading model checkpoint: {e}")
        import traceback
        traceback.print_exc()
        return None

def get_default_class_names():
    """Get default class names when no class_names.txt is available."""
    # Default class names: digits + uppercase letters + lowercase letters
    default_names = [str(i) for i in range(10)]  # 0-9
    default_names += [chr(i) for i in range(65, 91)]  # A-Z
    default_names += [chr(i) for i in range(97, 123)]  # a-z
    return default_names



In [ ]:
# --- Cell 3: Image Processing Functions ---
"""
## Image Processing Functions

These functions handle preprocessing of input images for character recognition.
They include operations for resizing, thresholding, and character segmentation.
"""

def preprocess_image(image, resize=True, target_size=(64, 64)):
    """
    Preprocess an image for inference.
    
    Args:
        image: Input image (numpy array or PIL Image)
        resize: Whether to resize the image
        target_size: Target size for resizing
        
    Returns:
        PIL.Image: Preprocessed image
    """
    # Convert to PIL Image if needed
    if isinstance(image, np.ndarray):
        if len(image.shape) == 3 and image.shape[2] == 4:  # RGBA
            image = Image.fromarray(image).convert('RGB')
        else:
            image = Image.fromarray(image)
    
    # Ensure grayscale
    gray_image = image.convert('L')
    
    # Resize if needed
    if resize:
        gray_image = gray_image.resize(target_size, Image.LANCZOS)
    
    return gray_image

def extract_characters(image, min_area=50, spacing=24):
    """
    Extract individual characters from an image containing text.
    
    Args:
        image: Input image (numpy array or PIL Image)
        min_area: Minimum contour area to consider as a character
        spacing: Spacing around characters when normalizing
        
    Returns:
        tuple: (extracted_chars, visualization_image)
    """
    # Convert to numpy array if needed
    if isinstance(image, Image.Image):
        image_np = np.array(image)
    else:
        image_np = image.copy()
    
    # Ensure grayscale
    if len(image_np.shape) == 3:
        gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    else:
        gray = image_np.copy()
    
    # Apply binary thresholding (black text on white background -> white text on black for findContours)
    _, binary_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours
    contours, _ = cv2.findContours(binary_inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter by area and sort left-to-right
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
    valid_contours = sorted(valid_contours, key=lambda cnt: cv2.boundingRect(cnt)[0])
    
    # Create visualization image
    vis_image = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    cv2.drawContours(vis_image, valid_contours, -1, (0, 255, 0), 2)
    
    # Extract and normalize individual characters
    extracted_chars = []
    
    for i, cnt in enumerate(valid_contours):
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Draw bounding box with index
        cv2.rectangle(vis_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(vis_image, str(i+1), (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Extract character ROI
        char_roi = gray[y:y+h, x:x+w]
        
        # Normalize size while maintaining aspect ratio
        target_size = 64 - spacing * 2
        
        if w > h:
            new_w = target_size
            new_h = int(h * new_w / w)
        else:
            new_h = target_size
            new_w = int(w * new_h / h)
        
        # Resize
        resized_char = cv2.resize(char_roi, (new_w, new_h), interpolation=cv2.INTER_AREA)
        
        # Create padded image (white background)
        padded_char = np.ones((64, 64), dtype=np.uint8) * 255
        
        # Calculate position to place resized image (centered)
        x_offset = (64 - new_w) // 2
        y_offset = (64 - new_h) // 2
        
        # Place resized image
        padded_char[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_char
        
        # Add to extracted characters
        extracted_chars.append({
            'image': padded_char,
            'bbox': (x, y, w, h),
            'index': i
        })
    
    return extracted_chars, vis_image

def recognize_characters(model, char_images, class_names, device=device):
    """
    Recognize characters using the model.
    
    Args:
        model: Trained model
        char_images: List of character images
        class_names: List of class names
        device: Device to run inference on
        
    Returns:
        list: Recognition results for each character
    """
    if not char_images:
        return []
    
    model.eval()
    results = []
    
    # Define transform for model input
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0) == 1 else x),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    for char_info in char_images:
        # Convert to PIL image
        char_pil = Image.fromarray(char_info['image'])
        
        # Apply transform
        char_tensor = transform(char_pil).unsqueeze(0).to(device)
        
        # Get prediction
        with torch.no_grad():
            outputs = model(char_tensor)
            probabilities = F.softmax(outputs, dim=1)
            confidence, predicted_idx = torch.max(probabilities, 1)
            
            predicted_class = class_names[predicted_idx.item()]
            confidence_score = confidence.item()
        
        # Store result
        results.append({
            'character': predicted_class,
            'confidence': confidence_score,
            'bbox': char_info['bbox'],
            'index': char_info['index']
        })
    
    return results

def create_visualization(original_image, char_results, char_images=None):
    """
    Create a visualization of the recognition results.
    
    Args:
        original_image: Original input image
        char_results: Character recognition results
        char_images: Extracted character images (optional)
        
    Returns:
        tuple: (result_image, extracted_grid)
    """
    # Convert original image to RGB for visualization
    if isinstance(original_image, Image.Image):
        original_np = np.array(original_image)
        if len(original_np.shape) == 2:  # Grayscale
            vis_image = cv2.cvtColor(original_np, cv2.COLOR_GRAY2RGB)
        else:
            vis_image = original_np.copy()
    else:
        if len(original_image.shape) == 2:  # Grayscale
            vis_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2RGB)
        else:
            vis_image = original_image.copy()
    
    # Draw bounding boxes and predictions
    for result in char_results:
        x, y, w, h = result['bbox']
        char = result['character']
        conf = result['confidence']
        
        # Draw bounding box
        cv2.rectangle(vis_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Draw prediction text
        text = f"{char} ({conf:.2f})"
        cv2.putText(vis_image, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # Create a grid of extracted characters if available
    extracted_grid = None
    if char_images is not None and len(char_images) > 0:
        # Determine grid size
        n_chars = len(char_images)
        grid_cols = min(8, n_chars)
        grid_rows = (n_chars + grid_cols - 1) // grid_cols
        
        # Create grid
        cell_size = 64
        grid_width = grid_cols * cell_size
        grid_height = grid_rows * cell_size
        grid = np.ones((grid_height, grid_width), dtype=np.uint8) * 255
        
        # Place images in grid
        for i, char_info in enumerate(char_images):
            if i >= grid_rows * grid_cols:
                break
                
            row = i // grid_cols
            col = i % grid_cols
            
            y_start = row * cell_size
            x_start = col * cell_size
            
            grid[y_start:y_start+cell_size, x_start:x_start+cell_size] = char_info['image']
        
        # Convert to RGB
        extracted_grid = cv2.cvtColor(grid, cv2.COLOR_GRAY2RGB)
        
        # Add character labels
        for i, result in enumerate(char_results):
            if i >= grid_rows * grid_cols:
                break
                
            row = i // grid_cols
            col = i % grid_cols
            
            x_text = col * cell_size + 5
            y_text = row * cell_size + 15
            
            cv2.putText(extracted_grid, result['character'], (x_text, y_text), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    return vis_image, extracted_grid

def enhance_image(image, enhancement_type):
    """
    Apply various image enhancements for better character recognition.
    
    Args:
        image: Input image (numpy array or PIL Image)
        enhancement_type: Type of enhancement to apply
        
    Returns:
        numpy.ndarray: Enhanced image
    """
    # Convert to numpy array if needed
    if isinstance(image, Image.Image):
        image_np = np.array(image)
    else:
        image_np = image.copy()
    
    # Ensure grayscale
    if len(image_np.shape) == 3:
        gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    else:
        gray = image_np.copy()
    
    if enhancement_type == "none":
        # No enhancement
        return gray
    
    elif enhancement_type == "threshold":
        # Otsu's thresholding
        _, enhanced = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return enhanced
    
    elif enhancement_type == "adaptive":
        # Adaptive thresholding
        enhanced = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY, 11, 2)
        return enhanced
    
    elif enhancement_type == "contrast":
        # Contrast enhancement
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)
        return enhanced
    
    elif enhancement_type == "denoise":
        # Denoising
        enhanced = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
        return enhanced
    
    elif enhancement_type == "sharpen":
        # Sharpening
        kernel = np.array([[-1, -1, -1],
                         [-1,  9, -1],
                         [-1, -1, -1]])
        enhanced = cv2.filter2D(gray, -1, kernel)
        return enhanced
    
    return gray  # Default fallback



In [ ]:
# --- Cell 4: Gradio Interface Components ---
"""
## Gradio Interface Components

These functions build the individual components of the Gradio interface.
Each component handles a specific aspect of the user interaction.
"""

def create_model_dropdown():
    """Create a dropdown for model selection."""
    # Find available models
    available_models = find_available_models()
    
    if not available_models:
        # Create a default dropdown with placeholder if no models found
        return gr.Dropdown(
            choices=["No models found - Please train models first"],
            value="No models found - Please train models first",
            label="Model Selection",
            info="No trained models were found. Please run the training notebook first."
        )
    
    # Create a list of choices from available models
    model_choices = list(available_models.keys())
    
    # Add descriptive labels
    model_labels = []
    for model_name in model_choices:
        # Try to get model info if available
        model_info_path = available_models[model_name].get('model_info_path')
        if model_info_path and os.path.exists(model_info_path):
            try:
                with open(model_info_path, 'r') as f:
                    info_text = f.read()
                    # Extract a short description
                    desc_line = [line for line in info_text.split('\n') if line.startswith('Description:')]
                    if desc_line:
                        description = desc_line[0].replace('Description:', '').strip()
                        model_labels.append(f"{model_name} - {description}")
                    else:
                        model_labels.append(model_name)
            except:
                model_labels.append(model_name)
        else:
            model_labels.append(model_name)
    
    # Create mapping of labels to model names
    label_to_model = {label: model for label, model in zip(model_labels, model_choices)}
    
    return gr.Dropdown(
        choices=model_labels,
        value=model_labels[0] if model_labels else None,
        label="Model Selection",
        info="Select a trained model to use for recognition",
    ), label_to_model

def create_model_info_box(label_to_model):
    """Create an info box for displaying model details."""
    # Function to load and display model info
    def load_model_info(model_label):
        if not model_label or "No models found" in model_label:
            return "No model selected or no models available."
        
        # Extract model name from label
        model_name = label_to_model.get(model_label, model_label)
        if not model_name:
            return "Invalid model selection."
        
        # Find available models to get info path
        available_models = find_available_models()
        if model_name not in available_models:
            return f"Model '{model_name}' not found in available models."
        
        model_info = available_models[model_name]
        
        # Check if model info file exists
        if not model_info.get('has_model_info'):
            # No info file, provide basic details
            checkpoint_files = model_info.get('checkpoint_files', [])
            class_names_path = model_info.get('class_names_path')
            
            # Count classes if available
            num_classes = "Unknown"
            if class_names_path and os.path.exists(class_names_path):
                try:
                    with open(class_names_path, 'r') as f:
                        class_names = [line.strip() for line in f if line.strip()]
                        num_classes = len(class_names)
                except:
                    pass
            
            info_text = f"Model: {model_name}\n"
            info_text += f"Number of classes: {num_classes}\n"
            info_text += f"Available checkpoints: {len(checkpoint_files)}\n"
            
            if checkpoint_files:
                # Try to extract some info from checkpoint
                latest_checkpoint = checkpoint_files[-1]
                try:
                    checkpoint = torch.load(latest_checkpoint, map_location='cpu')
                    if isinstance(checkpoint, dict):
                        if 'epoch' in checkpoint:
                            info_text += f"Trained epochs: {checkpoint['epoch']}\n"
                        if 'accuracy' in checkpoint or 'val_acc' in checkpoint:
                            acc = checkpoint.get('accuracy', checkpoint.get('val_acc', 'Unknown'))
                            info_text += f"Validation accuracy: {acc}\n"
                except:
                    pass
            
            return info_text
        else:
            # Read from info file
            info_path = model_info.get('model_info_path')
            try:
                with open(info_path, 'r') as f:
                    info_text = f.read()
                return info_text
            except:
                return f"Error reading model info for '{model_name}'."
    
    return gr.Textbox(
        value="Select a model to see details.",
        label="Model Information",
        lines=6,
        interactive=False
    ), load_model_info

def create_settings_accordion():
    """Create accordion with settings for character recognition."""
    with gr.Accordion("Recognition Settings", open=False) as settings_accordion:
        with gr.Row():
            checkpoint_type = gr.Radio(
                choices=["best", "final"],
                value="best",
                label="Checkpoint Type",
                info="Which model checkpoint to use"
            )
            min_area = gr.Slider(
                minimum=10,
                maximum=200,
                value=50,
                step=5,
                label="Minimum Character Area",
                info="Minimum area (in pixels) to consider as a character"
            )
        
        with gr.Row():
            enhancement = gr.Dropdown(
                choices=["none", "threshold", "adaptive", "contrast", "denoise", "sharpen"],
                value="none",
                label="Image Enhancement",
                info="Apply preprocessing to enhance image quality"
            )
            spacing = gr.Slider(
                minimum=0,
                maximum=32,
                value=24,
                step=4,
                label="Character Spacing",
                info="Spacing around characters when normalizing (pixels)"
            )
    
    return settings_accordion, checkpoint_type, min_area, enhancement, spacing

def create_debug_accordion():
    """Create accordion with debug information and visualization options."""
    with gr.Accordion("Debug & Visualization", open=False) as debug_accordion:
        with gr.Row():
            show_steps = gr.Checkbox(
                value=True,
                label="Show Processing Steps",
                info="Display intermediate processing steps"
            )
            show_confidence = gr.Checkbox(
                value=True,
                label="Show Confidence Scores",
                info="Display confidence scores for each character"
            )
        
        with gr.Row():
            debug_info = gr.Textbox(
                value="Debug information will appear here.",
                label="Debug Information",
                lines=5,
                interactive=False
            )
    
    return debug_accordion, show_steps, show_confidence, debug_info

def create_single_image_tab(label_to_model, load_model_info, settings):
    """Create tab for single image recognition (upload and draw)."""
    checkpoint_type, min_area, enhancement, spacing = settings
    
    with gr.Tab("Upload Image") as upload_tab:
        with gr.Row():
            with gr.Column(scale=3):
                image_input = gr.Image(
                    type="numpy",
                    label="Upload Image",
                    tool="select"
                )
                upload_button = gr.Button("Recognize Characters", variant="primary")
            
            with gr.Column(scale=5):
                with gr.Row():
                    result_image = gr.Image(
                        type="numpy",
                        label="Recognition Result"
                    )
                with gr.Row():
                    extracted_grid = gr.Image(
                        type="numpy",
                        label="Extracted Characters"
                    )
                with gr.Row():
                    result_text = gr.Textbox(
                        value="",
                        label="Recognized Text",
                        lines=1
                    )
    
    with gr.Tab("Draw Text") as draw_tab:
        with gr.Row():
            with gr.Column(scale=3):
                canvas_input = gr.Image(
                    type="numpy",
                    label="Draw Text",
                    tool="sketch",
                    height=180,
                    brush_radius=4,
                    source="canvas",
                    shape=(180, 600, 3),
                    invert_colors=False
                )
                draw_button = gr.Button("Recognize Characters", variant="primary")
                clear_button = gr.Button("Clear Canvas")
            
            with gr.Column(scale=5):
                with gr.Row():
                    draw_result_image = gr.Image(
                        type="numpy",
                        label="Recognition Result"
                    )
                with gr.Row():
                    draw_extracted_grid = gr.Image(
                        type="numpy",
                        label="Extracted Characters"
                    )
                with gr.Row():
                    draw_result_text = gr.Textbox(
                        value="",
                        label="Recognized Text",
                        lines=1
                    )
    
    with gr.Tab("Camera Capture") as camera_tab:
        with gr.Row():
            with gr.Column(scale=3):
                camera_input = gr.Image(
                    type="numpy",
                    label="Camera Capture",
                    sources=["webcam", "upload"],
                )
                camera_button = gr.Button("Recognize Characters", variant="primary")
            
            with gr.Column(scale=5):
                with gr.Row():
                    camera_result_image = gr.Image(
                        type="numpy",
                        label="Recognition Result"
                    )
                with gr.Row():
                    camera_extracted_grid = gr.Image(
                        type="numpy",
                        label="Extracted Characters"
                    )
                with gr.Row():
                    camera_result_text = gr.Textbox(
                        value="",
                        label="Recognized Text",
                        lines=1
                    )
    
    return {
        'upload': {
            'tab': upload_tab,
            'input': image_input,
            'button': upload_button,
            'result_image': result_image,
            'extracted_grid': extracted_grid,
            'result_text': result_text
        },
        'draw': {
            'tab': draw_tab,
            'input': canvas_input,
            'button': draw_button,
            'clear_button': clear_button,
            'result_image': draw_result_image,
            'extracted_grid': draw_extracted_grid,
            'result_text': draw_result_text
        },
        'camera': {
            'tab': camera_tab,
            'input': camera_input,
            'button': camera_button,
            'result_image': camera_result_image,
            'extracted_grid': camera_extracted_grid,
            'result_text': camera_result_text
        }
    }

def create_batch_processing_tab(label_to_model, load_model_info, settings):
    """Create tab for batch processing of multiple images."""
    checkpoint_type, min_area, enhancement, spacing = settings
    
    with gr.Tab("Batch Processing") as batch_tab:
        with gr.Row():
            with gr.Column(scale=2):
                batch_input = gr.File(
                    file_types=["image"],
                    file_count="multiple",
                    label="Upload Multiple Images"
                )
                batch_button = gr.Button("Process Batch", variant="primary")
            
            with gr.Column(scale=4):
                batch_results = gr.Gallery(
                    label="Batch Results",
                    columns=3,
                    height="auto",
                    object_fit="contain"
                )
        
        with gr.Row():
            batch_output = gr.Dataframe(
                headers=["Filename", "Recognized Text", "Confidence"],
                label="Batch Recognition Results",
                wrap=True,
                row_count=10,
                col_count=(3, "fixed")
            )
            
        with gr.Row():
            batch_download = gr.File(
                label="Download Results (CSV)",
                file_types=[".csv"],
                interactive=False
            )
    
    return {
        'tab': batch_tab,
        'input': batch_input,
        'button': batch_button,
        'results': batch_results,
        'output': batch_output,
        'download': batch_download
    }

def create_examples_section():
    """Create examples section with sample images for demonstration."""
    example_dir = "example_images"
    
    # Check if examples directory exists
    if not os.path.exists(example_dir):
        os.makedirs(example_dir, exist_ok=True)
        
        # Create some example images if none exist
        try:
            # Example 1: Simple text
            example1 = np.ones((100, 300), dtype=np.uint8) * 255
            cv2.putText(example1, "Hello123", (50, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)
            cv2.imwrite(os.path.join(example_dir, "example1.png"), example1)
            
            # Example 2: Mixed characters
            example2 = np.ones((100, 300), dtype=np.uint8) * 255
            cv2.putText(example2, "Abc123", (50, 60), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1.5, (0, 0, 0), 2)
            cv2.imwrite(os.path.join(example_dir, "example2.png"), example2)
            
            # Example 3: Spaced characters
            example3 = np.ones((100, 300), dtype=np.uint8) * 255
            cv2.putText(example3, "A B C 1 2 3", (20, 60), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)
            cv2.imwrite(os.path.join(example_dir, "example3.png"), example3)
        except:
            print("Could not create example images")
    
    # Find existing examples
    example_images = []
    for ext in ['*.png', '*.jpg', '*.jpeg']:
        example_images.extend(glob(os.path.join(example_dir, ext)))
    
    return gr.Examples(
        examples=example_images,
        inputs=["image"],
        label="Example Images",
        examples_per_page=5
    )



In [ ]:
# --- Cell 5: Main Recognition Functions ---
"""
## Main Recognition Functions

These functions handle the core recognition logic for different input methods.
They coordinate loading models, processing images, and generating results.
"""

def recognize_image(image, model_label, checkpoint_type, min_area, enhancement_type, spacing, 
                  show_steps, show_confidence, debug_info, label_to_model):
    """
    Process an uploaded image and recognize characters.
    
    Args:
        image: Input image from Gradio
        model_label: Selected model from dropdown
        checkpoint_type: Type of checkpoint to use
        min_area: Minimum contour area to consider
        enhancement_type: Type of image enhancement to apply
        spacing: Character spacing for normalization
        show_steps: Whether to show processing steps
        show_confidence: Whether to show confidence scores
        debug_info: Debug info text box
        label_to_model: Mapping from labels to model names
        
    Returns:
        tuple: (result_image, extracted_grid, result_text)
    """
    debug_text = "Processing image...\n"
    
    # Check if an image was provided
    if image is None:
        debug_text += "ERROR: No image provided."
        return None, None, "", debug_text
    
    # Extract model name from label
    model_name = label_to_model.get(model_label, model_label)
    debug_text += f"Using model: {model_name}\n"
    
    try:
        # Find available models
        available_models = find_available_models()
        
        if model_name not in available_models:
            debug_text += f"ERROR: Model '{model_name}' not found."
            return None, None, "", debug_text
        
        model_info = available_models[model_name]
        
        # Determine checkpoint path
        checkpoint_path = None
        
        if checkpoint_type == "best":
            # Look for best_model.pth
            best_paths = [cp for cp in model_info['checkpoint_files'] if 'best' in os.path.basename(cp).lower()]
            if best_paths:
                checkpoint_path = best_paths[0]
            else:
                debug_text += "WARNING: No 'best' checkpoint found, using first available checkpoint.\n"
                checkpoint_path = model_info['checkpoint_files'][0]
        else:  # "final"
            # Look for final_model.pth
            final_paths = [cp for cp in model_info['checkpoint_files'] if 'final' in os.path.basename(cp).lower()]
            if final_paths:
                checkpoint_path = final_paths[0]
            else:
                debug_text += "WARNING: No 'final' checkpoint found, using first available checkpoint.\n"
                checkpoint_path = model_info['checkpoint_files'][0]
        
        # Load class names
        if model_info['has_class_names']:
            class_names = load_class_names(model_info['class_names_path'])
        else:
            debug_text += "WARNING: No class_names.txt found, using default class names.\n"
            class_names = get_default_class_names()
        
        if class_names is None:
            debug_text += "ERROR: Failed to load class names."
            return None, None, "", debug_text
        
        num_classes = len(class_names)
        
        # Load model
        model = load_model_checkpoint(checkpoint_path, model_name, num_classes, device)
        
        if model is None:
            debug_text += "ERROR: Failed to load model."
            return None, None, "", debug_text
        
        # Preprocess image
        debug_text += "Preprocessing image...\n"
        
        # Apply enhancement if requested
        if enhancement_type != "none":
            debug_text += f"Applying {enhancement_type} enhancement...\n"
            enhanced_image = enhance_image(image, enhancement_type)
        else:
            enhanced_image = image
        
        # Extract characters
        debug_text += "Extracting characters...\n"
        char_images, vis_image = extract_characters(enhanced_image, min_area=min_area, spacing=spacing)
        
        if not char_images:
            debug_text += "WARNING: No characters detected in the image."
            return vis_image, None, "", debug_text
        
        debug_text += f"Found {len(char_images)} potential characters.\n"
        
        # Recognize characters
        debug_text += "Recognizing characters...\n"
        char_results = recognize_characters(model, char_images, class_names, device)
        
        # Create result text
        result_text = ''.join(result['character'] for result in char_results)
        
        # Create visualization
        debug_text += "Creating visualization...\n"
        result_image, extracted_grid = create_visualization(enhanced_image, char_results, char_images)
        
        debug_text += f"Recognition complete. Detected text: {result_text}\n"
        
        return result_image, extracted_grid, result_text, debug_text
    
    except Exception as e:
        import traceback
        debug_text += f"ERROR: {str(e)}\n"
        debug_text += traceback.format_exc()
        return None, None, "", debug_text

def process_batch_images(batch_files, model_label, checkpoint_type, min_area, enhancement_type, 
                        spacing, label_to_model):
    """
    Process a batch of images for character recognition.
    
    Args:
        batch_files: List of uploaded files
        model_label: Selected model from dropdown
        checkpoint_type: Type of checkpoint to use
        min_area: Minimum contour area to consider
        enhancement_type: Type of image enhancement to apply
        spacing: Character spacing for normalization
        label_to_model: Mapping from labels to model names
        
    Returns:
        tuple: (gallery_items, results_df, csv_file)
    """
    if not batch_files:
        return [], None, None
    
    # Extract model name from label
    model_name = label_to_model.get(model_label, model_label)
    
    try:
        # Find available models
        available_models = find_available_models()
        
        if model_name not in available_models:
            return [], None, None
        
        model_info = available_models[model_name]
        
        # Determine checkpoint path
        checkpoint_path = None
        
        if checkpoint_type == "best":
            # Look for best_model.pth
            best_paths = [cp for cp in model_info['checkpoint_files'] if 'best' in os.path.basename(cp).lower()]
            if best_paths:
                checkpoint_path = best_paths[0]
            else:
                checkpoint_path = model_info['checkpoint_files'][0]
        else:  # "final"
            # Look for final_model.pth
            final_paths = [cp for cp in model_info['checkpoint_files'] if 'final' in os.path.basename(cp).lower()]
            if final_paths:
                checkpoint_path = final_paths[0]
            else:
                checkpoint_path = model_info['checkpoint_files'][0]
        
        # Load class names
        if model_info['has_class_names']:
            class_names = load_class_names(model_info['class_names_path'])
        else:
            class_names = get_default_class_names()
        
        if class_names is None:
            return [], None, None
        
        num_classes = len(class_names)
        
        # Load model
        model = load_model_checkpoint(checkpoint_path, model_name, num_classes, device)
        
        if model is None:
            return [], None, None
        
        # Process each image
        gallery_items = []
        results = []
        
        for file in batch_files:
            try:
                # Load image
                image = Image.open(file.name)
                filename = os.path.basename(file.name)
                
                # Apply enhancement if requested
                if enhancement_type != "none":
                    enhanced_image = enhance_image(image, enhancement_type)
                else:
                    enhanced_image = image
                
                # Extract characters
                char_images, vis_image = extract_characters(enhanced_image, min_area=min_area, spacing=spacing)
                
                if not char_images:
                    # No characters detected
                    results.append({
                        'filename': filename,
                        'text': "No characters detected",
                        'confidence': 0.0
                    })
                    
                    # Add to gallery
                    gallery_items.append((file.name, vis_image))
                    continue
                
                # Recognize characters
                char_results = recognize_characters(model, char_images, class_names, device)
                
                # Create result text
                result_text = ''.join(result['character'] for result in char_results)
                
                # Calculate average confidence
                avg_confidence = sum(result['confidence'] for result in char_results) / len(char_results)
                
                # Create visualization
                result_image, _ = create_visualization(enhanced_image, char_results, char_images)
                
                # Add to results
                results.append({
                    'filename': filename,
                    'text': result_text,
                    'confidence': avg_confidence
                })
                
                # Add to gallery
                gallery_items.append((file.name, result_image))
            
            except Exception as e:
                # Add error result
                results.append({
                    'filename': os.path.basename(file.name),
                    'text': f"Error: {str(e)}",
                    'confidence': 0.0
                })
        
        # Create DataFrame for display
        results_df = [(r['filename'], r['text'], f"{r['confidence']:.2f}") for r in results]
        
        # Create CSV file
        csv_content = "Filename,Recognized Text,Confidence\n"
        for r in results:
            csv_content += f"\"{r['filename']}\",\"{r['text']}\",{r['confidence']:.4f}\n"
        
        csv_file = os.path.join("gradio_results", "batch", "batch_results.csv")
        os.makedirs(os.path.dirname(csv_file), exist_ok=True)
        
        with open(csv_file, 'w') as f:
            f.write(csv_content)
        
        return gallery_items, results_df, csv_file
    
    except Exception as e:
        import traceback
        print(f"Error in batch processing: {str(e)}")
        print(traceback.format_exc())
        return [], None, None



In [ ]:
# --- Cell 6: Build Complete Gradio Interface ---
"""
## Build Complete Gradio Interface

This section assembles all the components into a complete interactive application.
It defines the layout, sets up event handlers, and launches the interface.
"""

def build_interface():
    """Build and launch the complete Gradio interface."""
    with gr.Blocks(theme=gr.themes.Soft(), title="Handwritten Character Recognition") as app:
        gr.Markdown("# Handwritten Character Recognition")
        gr.Markdown("""
        This application recognizes handwritten characters using deep learning models. 
        You can upload an image, draw text, or capture from your camera.
        
        Select a model from the dropdown and use the tabs to access different input methods.
        """)
        
        # Create model selection components
        model_dropdown, label_to_model = create_model_dropdown()
        model_info, load_model_info = create_model_info_box(label_to_model)
        
        # Create settings accordion
        settings_accordion, checkpoint_type, min_area, enhancement, spacing = create_settings_accordion()
        
        # Create debug accordion
        debug_accordion, show_steps, show_confidence, debug_info = create_debug_accordion()
        
        # Create tabs for different input methods
        with gr.Tabs():
            # Single image tabs (upload, draw, camera)
            single_tabs = create_single_image_tab(
                label_to_model,
                load_model_info,
                (checkpoint_type, min_area, enhancement, spacing)
            )
            
            # Batch processing tab
            batch_tab = create_batch_processing_tab(
                label_to_model,
                load_model_info,
                (checkpoint_type, min_area, enhancement, spacing)
            )
        
        # Add examples section
        examples = create_examples_section()
        
        # Set up event handlers
        
        # Update model info when selection changes
        model_dropdown.change(
            fn=load_model_info,
            inputs=[model_dropdown],
            outputs=[model_info]
        )
        
        # Upload tab
        single_tabs['upload']['button'].click(
            fn=recognize_image,
            inputs=[
                single_tabs['upload']['input'],
                model_dropdown,
                checkpoint_type,
                min_area,
                enhancement,
                spacing,
                show_steps,
                show_confidence,
                debug_info,
                gr.State(label_to_model)
            ],
            outputs=[
                single_tabs['upload']['result_image'],
                single_tabs['upload']['extracted_grid'],
                single_tabs['upload']['result_text'],
                debug_info
            ]
        )
        
        # Draw tab
        single_tabs['draw']['button'].click(
            fn=recognize_image,
            inputs=[
                single_tabs['draw']['input'],
                model_dropdown,
                checkpoint_type,
                min_area,
                enhancement,
                spacing,
                show_steps,
                show_confidence,
                debug_info,
                gr.State(label_to_model)
            ],
            outputs=[
                single_tabs['draw']['result_image'],
                single_tabs['draw']['extracted_grid'],
                single_tabs['draw']['result_text'],
                debug_info
            ]
        )
        
        # Clear drawing canvas
        single_tabs['draw']['clear_button'].click(
            fn=lambda: (None, None, "", "Canvas cleared."),
            inputs=[],
            outputs=[
                single_tabs['draw']['result_image'],
                single_tabs['draw']['extracted_grid'],
                single_tabs['draw']['result_text'],
                debug_info
            ]
        )
        
        # Camera tab
        single_tabs['camera']['button'].click(
            fn=recognize_image,
            inputs=[
                single_tabs['camera']['input'],
                model_dropdown,
                checkpoint_type,
                min_area,
                enhancement,
                spacing,
                show_steps,
                show_confidence,
                debug_info,
                gr.State(label_to_model)
            ],
            outputs=[
                single_tabs['camera']['result_image'],
                single_tabs['camera']['extracted_grid'],
                single_tabs['camera']['result_text'],
                debug_info
            ]
        )
        
        # Batch tab
        batch_tab['button'].click(
            fn=process_batch_images,
            inputs=[
                batch_tab['input'],
                model_dropdown,
                checkpoint_type,
                min_area,
                enhancement,
                spacing,
                gr.State(label_to_model)
            ],
            outputs=[
                batch_tab['results'],
                batch_tab['output'],
                batch_tab['download']
            ]
        )
        
        # Connect examples to upload tab
        examples.dataset.click(
            fn=lambda x: x,
            inputs=[examples.components[0]],
            outputs=[single_tabs['upload']['input']]
        )
        
        # Add CSS for styling
        gr.Markdown("""
        <style>
        .gradio-container {
            max-width: 1200px !important;
            margin-left: auto !important;
            margin-right: auto !important;
        }
        
        .gr-button.gr-button-lg.gr-button-primary {
            background-color: #2980b9 !important;
        }
        
        .gr-button.gr-button-lg.gr-button-secondary {
            background-color: #95a5a6 !important;
        }
        
        .footer {
            margin-top: 20px;
            text-align: center;
            font-size: 0.8em;
            color: #7f8c8d;
        }
        </style>
        """)
        
        # Add footer
        gr.Markdown("""
        <div class="footer">
        <p>Handwritten Character Recognition Application</p>
        <p>Created with Gradio, PyTorch, and OpenCV</p>
        </div>
        """)
    
    return app



In [ ]:
# --- Cell 7: Launch Gradio Application ---
"""
## Launch Gradio Application

This section launches the Gradio application with appropriate settings.
It handles startup configurations and provides a public share link if requested.
"""

def launch_application(share=True, debug=False):
    """
    Launch the Gradio application.
    
    Args:
        share: Whether to create a public share link
        debug: Whether to enable debug mode
    """
    # Find available models first
    available_models = find_available_models()
    
    if not available_models:
        print("WARNING: No trained models found. Please run the training notebook first.")
        print("The application will start, but you won't be able to perform recognition.")
    else:
        print(f"Found {len(available_models)} trained models:")
        for model_name in available_models:
            print(f"  - {model_name}")
    
    # Build the interface
    app = build_interface()
    
    # Launch with specified settings
    app.launch(
        share=share,
        debug=debug,
        server_name="0.0.0.0",  # Listen on all network interfaces
        server_port=7860,
        inbrowser=True,
        show_api=False,
        max_threads=40,
    )

# Example usage
if __name__ == "__main__":
    # Launch with share=True to generate a public URL
    launch_application(share=True, debug=False)



In [ ]:
# --- Cell 8: Run Gradio Application ---
"""
## Run Gradio Application

This is where you run the Gradio application.
Uncomment and modify the code below to launch the application with your desired settings.
"""

# Launch the application
# launch_application(share=True)  # Set share=True to get a public URL

print("This notebook is ready to launch the Gradio application for handwritten character recognition.")
print("Execute this cell to start the application.")